## Libraries

In [32]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights, run_weighting_iteration


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(biden_vote, trump_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 7.5
    TIME_SHIFT_ERROR = 1.0
    N_SIMS = 100000
    
    biden_moe = margin_of_error(n=n, p=biden_vote/100, interval_size=interval)
    trump_moe = margin_of_error(n=n, p=trump_vote/100, interval_size=interval)
    undecided = (100 - biden_vote - trump_vote) / 2

    biden_mean = biden_vote + undecided * 0.25
    biden_raw_moe = biden_moe * 100
    biden_allocate_undecided = undecided * 0.4
    biden_margin = biden_raw_moe + biden_allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    trump_mean = trump_vote + undecided * 0.25
    trump_raw_moe = trump_moe * 100
    trump_allocate_undecided = undecided * 0.4
    trump_margin = trump_raw_moe + trump_allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    biden_sigma = biden_margin / 100 / normed_sigma
    biden_sims = np.random.normal(biden_mean / 100, biden_sigma, N_SIMS)
    
    trump_sigma = trump_margin / 100 / normed_sigma
    trump_sims = np.random.normal(trump_mean / 100, trump_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] > sim[1] for sim in zip(biden_sims, trump_sims)]) / N_SIMS
    
    low, high = np.percentile(biden_sims - trump_sims, [20, 80]) * 100
    
    return {'mean': biden_mean - trump_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': biden_raw_moe + trump_raw_moe,
            'margin': (biden_margin + trump_margin) / 2,
            'sigma': (biden_sigma + trump_sigma) / 2,
            'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result Biden {} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) (Biden {} likely to win)').format(mean,
                                                           first,
                                                           second,
                                                           n,
                                                           raw_moe,
                                                           margin,
                                                           sigma,
                                                           chance_pass))
    print(('Biden {} (80% CI: {} to {}) ({} Biden)').format(mean,
                                                            first,
                                                            second,
                                                            chance_pass))
    print('-')
    



def calc_result_sen(dem_vote, rep_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 5.0
    TIME_SHIFT_ERROR = 0.5
    N_SIMS = 100000
    
    dem_moe = margin_of_error(n=n, p=dem_vote/100, interval_size=interval)
    rep_moe = margin_of_error(n=n, p=rep_vote/100, interval_size=interval)
    undecided = 100 - dem_vote - rep_vote

    dem_mean = dem_vote + undecided * 0.25
    dem_raw_moe = dem_moe * 100
    dem_allocate_undecided = undecided * 0.4
    dem_margin = dem_raw_moe + dem_allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    rep_mean = rep_vote + undecided * 0.25
    rep_raw_moe = rep_moe * 100
    rep_allocate_undecided = undecided * 0.4
    rep_margin = rep_raw_moe + rep_allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    dem_sigma = dem_margin / 100 / normed_sigma
    dem_sims = np.random.normal(dem_mean / 100, dem_sigma, N_SIMS)
    
    rep_sigma = rep_margin / 100 / normed_sigma
    rep_sims = np.random.normal(rep_mean / 100, rep_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] > sim[1] for sim in zip(dem_sims, rep_sims)]) / N_SIMS
    
    low, high = np.percentile(dem_sims - rep_sims, [20, 80]) * 100
    
    return {'mean': dem_mean - rep_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': dem_raw_moe + rep_raw_moe,
            'margin': (dem_margin + rep_margin) / 2,
            'sigma': (dem_sigma + rep_sigma) / 2,
            'chance_pass': chance_pass}


def print_result_sen(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result Dem Sen {} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) (Dem Sen {} likely to win)').format(mean,
                                                             first,
                                                             second,
                                                             n,
                                                             raw_moe,
                                                             margin,
                                                             sigma,
                                                             chance_pass))
    print(('Dem {} (80% CI: {} to {}) ({} Dem)').format(mean,
                                                        first,
                                                        second,
                                                        chance_pass))

    print('-')

## Load Processed Data

In [33]:
survey = pd.read_csv('responses_processed_national_weighted.csv').fillna('Not presented')

## State Presidential Models

In [34]:
POTUS_CENSUS = {'Alabama': {'Hillary Clinton': 0.3436, 'Donald Trump': 0.6208},
                'Alaska': {'Hillary Clinton': 0.3655, 'Donald Trump': 0.5128},
                'Arizona': {'Hillary Clinton': 0.4513, 'Donald Trump': 0.4867},
                'Arkansas': {'Hillary Clinton': 0.3365, 'Donald Trump': 0.6057},
                'California': {'Hillary Clinton': 0.6173, 'Donald Trump': 0.3162},
                'Colorado': {'Hillary Clinton': 0.4816, 'Donald Trump': 0.4325},
                'Connecticut': {'Hillary Clinton': 0.5457, 'Donald Trump': 0.4093},
                'Delaware': {'Hillary Clinton': 0.531, 'Donald Trump': 0.417},
                'Washington DC': {'Hillary Clinton': 0.905, 'Donald Trump': 0.016},
                'Florida': {'Hillary Clinton': 0.478, 'Donald Trump': 0.490},
                'Georgia': {'Hillary Clinton': 0.456, 'Donald Trump': 0.508},
                'Hawaii': {'Hillary Clinton': 0.622, 'Donald Trump': 0.300},
                'Idaho': {'Hillary Clinton': 0.275, 'Donald Trump': 0.593},
                'Illinois': {'Hillary Clinton': 0.558, 'Donald Trump': 0.379},
                'Indiana': {'Hillary Clinton': 0.379, 'Donald Trump': 0.511},
                'Iowa': {'Hillary Clinton': 0.417, 'Donald Trump': 0.512},
                'Kansas': {'Hillary Clinton': 0.361, 'Donald Trump': 0.567},
                'Kentucky': {'Hillary Clinton': 0.327, 'Donald Trump': 0.625},
                'Louisiana': {'Hillary Clinton': 0.385, 'Donald Trump': 0.581},
                'Maine': {'Hillary Clinton': 0.478, 'Donald Trump': 0.449},
                'Maryland': {'Hillary Clinton': 0.603, 'Donald Trump': 0.339},
                'Massachusetts': {'Hillary Clinton': 0.600, 'Donald Trump': 0.328},
                'Michigan': {'Hillary Clinton': 0.473, 'Donald Trump': 0.475},
                'Minnesota': {'Hillary Clinton': 0.464, 'Donald Trump': 0.449},
                'Mississippi': {'Hillary Clinton': 0.401, 'Donald Trump': 0.579},
                'Missouri': {'Hillary Clinton': 0.401, 'Donald Trump': 0.579},
                'Montana': {'Hillary Clinton': 0.381, 'Donald Trump': 0.562},
                'Nebraska': {'Hillary Clinton': 0.337, 'Donald Trump': 0.588},
                'Nevada': {'Hillary Clinton': 0.479, 'Donald Trump': 0.455},
                'New Hampshire': {'Hillary Clinton': 0.470, 'Donald Trump': 0.466},
                'New Jersey': {'Hillary Clinton': 0.555, 'Donald Trump': 0.414},
                'New Mexico': {'Hillary Clinton': 0.483, 'Donald Trump': 0.404},
                'New York': {'Hillary Clinton': 0.590, 'Donald Trump': 0.365},
                'North Carolina': {'Hillary Clinton': 0.462, 'Donald Trump': 0.498},
                'North Dakota': {'Hillary Clinton': 0.272, 'Donald Trump': 0.630},
                'Ohio': {'Hillary Clinton': 0.436, 'Donald Trump': 0.517},
                'Oklahoma': {'Hillary Clinton': 0.289, 'Donald Trump': 0.653},
                'Oregon': {'Hillary Clinton': 0.501, 'Donald Trump': 0.391},
                'Pennsylvania': {'Hillary Clinton': 0.475, 'Donald Trump': 0.481},
                'Rhode Island': {'Hillary Clinton': 0.544, 'Donald Trump': 0.389},
                'South Carolina': {'Hillary Clinton': 0.407, 'Donald Trump': 0.549},
                'South Dakota': {'Hillary Clinton': 0.317, 'Donald Trump': 0.615},
                'Tennessee': {'Hillary Clinton': 0.347, 'Donald Trump': 0.607},
                'Texas': {'Hillary Clinton': 0.432, 'Donald Trump': 0.522},
                'Utah': {'Hillary Clinton': 0.275, 'Donald Trump': 0.454},
                'Vermont': {'Hillary Clinton': 0.567, 'Donald Trump': 0.303},
                'Virginia': {'Hillary Clinton': 0.497, 'Donald Trump': 0.444},
                'Washington': {'Hillary Clinton': 0.525, 'Donald Trump': 0.368},
                'West Virginia': {'Hillary Clinton': 0.264, 'Donald Trump': 0.685},
                'Wisconsin': {'Hillary Clinton': 0.465, 'Donald Trump': 0.472},
                'Wyoming': {'Hillary Clinton': 0.216, 'Donald Trump': 0.674 }}

for state in POTUS_CENSUS.keys():
    print('## {} ##'.format(state.upper()))
    state_survey = survey.copy()
    potus_census = {'vote2016': POTUS_CENSUS[state].copy()}
    potus_census['vote2016']['Other'] = 1 - potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
    output = run_weighting_iteration(state_survey, census=potus_census, weigh_on=['vote2016'], verbose=0)
    potus_weights = output['weights']['vote2016']
    potus_weights = state_survey['vote2016'].astype(str).replace(potus_weights)
    state_survey['weight'] = normalize_weights(state_survey['weight'] * potus_weights)
    state_survey['lv_weight'] = normalize_weights(state_survey['weight'] * state_survey['lv_index'])

    options = ['Donald Trump', 'Hillary Clinton', 'Other']
    survey_ = state_survey.loc[state_survey['vote2016'].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_['vote2016'].value_counts(normalize=True) * survey_.groupby('vote2016')['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    raw_result = potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
    print('Raw result: {}'.format(np.round(raw_result * 100, 1)))
    print(votes)
    print('-')
    print_result(**calc_result(biden_vote=votes['Hillary Clinton'],
                               trump_vote=votes['Donald Trump'],
                               n=lv_weighted_n))

    options = ['Joe Biden, the Democrat', 'Donald Trump, the Republican', 'Another candidate', 'Not decided']
    survey_ = state_survey.loc[state_survey['vote_trump_biden'].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

    votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print('-')
    print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                               trump_vote=votes['Donald Trump, the Republican'],
                               n=lv_weighted_n))
    print('-')

## ALABAMA ##
Raw result: -27.7
Donald Trump       62.429369
Hillary Clinton    33.968809
Other               3.601822
dtype: float64
-
Result Biden -28.5 (80% CI: -38.6 to -18.3) (Weighted N=1232) (raw_moe=3.5pts, margin=11.0pts, sigma=8.6pts) (Biden 0.9% likely to win)
Biden -28.5 (80% CI: -38.6 to -18.3) (0.9% Biden)
-
Joe Biden, the Democrat         41.582205
Donald Trump, the Republican    51.858405
Another candidate                3.074509
Not decided                      3.484882
dtype: float64
-
Result Biden -10.3 (80% CI: -21.0 to 0.5) (Weighted N=1232) (raw_moe=3.6pts, margin=11.6pts, sigma=9.1pts) (Biden 21.0% likely to win)
Biden -10.3 (80% CI: -21.0 to 0.5) (21.0% Biden)
-
-
## ALASKA ##
Raw result: -14.7
Donald Trump       51.560631
Hillary Clinton    36.128300
Other              12.311069
dtype: float64
-
Result Biden -15.4 (80% CI: -27.2 to -3.6) (Weighted N=1285) (raw_moe=3.5pts, margin=12.7pts, sigma=9.9pts) (Biden 13.6% likely to win)
Biden -15.4 (80% CI: -27.2 to -3

Result Biden 17.1 (80% CI: 6.5 to 27.8) (Weighted N=1347) (raw_moe=3.4pts, margin=11.5pts, sigma=9.0pts) (Biden 91.1% likely to win)
Biden 17.1 (80% CI: 6.5 to 27.8) (91.1% Biden)
-
Joe Biden, the Democrat         56.633918
Donald Trump, the Republican    36.004685
Another candidate                3.736882
Not decided                      3.624515
dtype: float64
-
Result Biden 20.6 (80% CI: 9.8 to 31.4) (Weighted N=1347) (raw_moe=3.4pts, margin=11.7pts, sigma=9.1pts) (Biden 94.5% likely to win)
Biden 20.6 (80% CI: 9.8 to 31.4) (94.5% Biden)
-
-
## INDIANA ##
Raw result: -13.2
Donald Trump       51.395128
Hillary Clinton    37.474012
Other              11.130859
dtype: float64
-
Result Biden -13.9 (80% CI: -25.5 to -2.3) (Weighted N=1291) (raw_moe=3.5pts, margin=12.5pts, sigma=9.7pts) (Biden 15.5% likely to win)
Biden -13.9 (80% CI: -25.5 to -2.3) (15.5% Biden)
-
Joe Biden, the Democrat         45.891706
Donald Trump, the Republican    45.859312
Another candidate                4.586525

Result Biden -18.6 (80% CI: -28.5 to -8.8) (Weighted N=1254) (raw_moe=3.6pts, margin=10.7pts, sigma=8.3pts) (Biden 5.6% likely to win)
Biden -18.6 (80% CI: -28.5 to -8.8) (5.6% Biden)
-
Joe Biden, the Democrat         45.004165
Donald Trump, the Republican    48.733679
Another candidate                2.790726
Not decided                      3.471430
dtype: float64
-
Result Biden -3.7 (80% CI: -14.4 to 7.1) (Weighted N=1254) (raw_moe=3.6pts, margin=11.6pts, sigma=9.0pts) (Biden 38.6% likely to win)
Biden -3.7 (80% CI: -14.4 to 7.1) (38.6% Biden)
-
-
## MONTANA ##
Raw result: -18.1
Donald Trump       56.544851
Hillary Clinton    37.685272
Other               5.769877
dtype: float64
-
Result Biden -18.9 (80% CI: -29.5 to -8.2) (Weighted N=1272) (raw_moe=3.5pts, margin=11.4pts, sigma=8.9pts) (Biden 6.8% likely to win)
Biden -18.9 (80% CI: -29.5 to -8.2) (6.8% Biden)
-
Joe Biden, the Democrat         44.637860
Donald Trump, the Republican    48.292565
Another candidate                3.52

Result Biden 10.2 (80% CI: -1.4 to 21.8) (Weighted N=1343) (raw_moe=3.5pts, margin=12.4pts, sigma=9.7pts) (Biden 77.2% likely to win)
Biden 10.2 (80% CI: -1.4 to 21.8) (77.2% Biden)
-
Joe Biden, the Democrat         53.993563
Donald Trump, the Republican    37.691716
Another candidate                4.611593
Not decided                      3.703127
dtype: float64
-
Result Biden 16.3 (80% CI: 5.2 to 27.4) (Weighted N=1343) (raw_moe=3.4pts, margin=11.9pts, sigma=9.3pts) (Biden 89.2% likely to win)
Biden 16.3 (80% CI: 5.2 to 27.4) (89.2% Biden)
-
-
## PENNSYLVANIA ##
Raw result: -0.6
Donald Trump       48.476570
Hillary Clinton    47.061997
Other               4.461434
dtype: float64
-
Result Biden -1.4 (80% CI: -11.8 to 8.9) (Weighted N=1312) (raw_moe=3.5pts, margin=11.2pts, sigma=8.7pts) (Biden 45.5% likely to win)
Biden -1.4 (80% CI: -11.8 to 8.9) (45.5% Biden)
-
Joe Biden, the Democrat         50.578707
Donald Trump, the Republican    42.558155
Another candidate                3.3113

Result Biden -1.5 (80% CI: -12.2 to 9.3) (Weighted N=1318) (raw_moe=3.5pts, margin=11.5pts, sigma=9.0pts) (Biden 45.3% likely to win)
Biden -1.5 (80% CI: -12.2 to 9.3) (45.3% Biden)
-
Joe Biden, the Democrat         50.407248
Donald Trump, the Republican    42.314490
Another candidate                3.688078
Not decided                      3.590185
dtype: float64
-
Result Biden 8.1 (80% CI: -2.8 to 19.0) (Weighted N=1318) (raw_moe=3.5pts, margin=11.7pts, sigma=9.1pts) (Biden 73.3% likely to win)
Biden 8.1 (80% CI: -2.8 to 19.0) (73.3% Biden)
-
-
## WYOMING ##
Raw result: -45.8
Donald Trump       67.601776
Hillary Clinton    21.298151
Other              11.100074
dtype: float64
-
Result Biden -46.3 (80% CI: -57.7 to -34.9) (Weighted N=1164) (raw_moe=3.3pts, margin=12.4pts, sigma=9.6pts) (Biden <0.1% likely to win)
Biden -46.3 (80% CI: -57.7 to -34.9) (<0.1% Biden)
-
Joe Biden, the Democrat         35.033901
Donald Trump, the Republican    56.864071
Another candidate                4.49

## Senate Models

In [35]:
for state in POTUS_CENSUS.keys():
    print('## {} ##'.format(state.upper()))
    state_survey = survey.copy()
    potus_census = {'vote2016': POTUS_CENSUS[state].copy()}
    potus_census['vote2016']['Other'] = 1 - potus_census['vote2016']['Hillary Clinton'] - potus_census['vote2016']['Donald Trump']
    output = run_weighting_iteration(state_survey, census=potus_census, weigh_on=['vote2016'], verbose=0)
    potus_weights = output['weights']['vote2016']
    potus_weights = state_survey['vote2016'].astype(str).replace(potus_weights)
    state_survey['weight'] = normalize_weights(state_survey['weight'] * potus_weights)
    state_survey['lv_weight'] = normalize_weights(state_survey['weight'] * state_survey['lv_index'])

    options = ['A Democratic candidate', 'A Republican candidate', 'Another candidate', 'Not decided']
    survey_ = state_survey.loc[state_survey['vote_senate'].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

    votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print('-')
    print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                       rep_vote=votes['A Republican candidate'],
                                       n=lv_weighted_n))
    print('-')

## ALABAMA ##
A Democratic candidate    37.243159
A Republican candidate    51.746655
Another candidate          1.699965
Not decided                9.310221
dtype: float64
-
Result Dem Sen -14.5 (80% CI: -25.4 to -3.5) (Weighted N=1164) (raw_moe=3.7pts, margin=11.7pts, sigma=9.2pts) (Dem Sen 13.3% likely to win)
Dem -14.5 (80% CI: -25.4 to -3.5) (13.3% Dem)
-
-
## ALASKA ##
A Democratic candidate    40.627153
A Republican candidate    47.731694
Another candidate          2.004306
Not decided                9.636847
dtype: float64
-
Result Dem Sen -7.1 (80% CI: -18.2 to 4.1) (Weighted N=1164) (raw_moe=3.7pts, margin=12.0pts, sigma=9.4pts) (Dem Sen 29.8% likely to win)
Dem -7.1 (80% CI: -18.2 to 4.1) (29.8% Dem)
-
-
## ARIZONA ##
A Democratic candidate    44.940957
A Republican candidate    44.005564
Another candidate          1.856676
Not decided                9.196803
dtype: float64
-
Result Dem Sen 0.9 (80% CI: -10.0 to 12.0) (Weighted N=1164) (raw_moe=3.7pts, margin=11.8pts, sigma=

A Democratic candidate    46.382774
A Republican candidate    42.390116
Another candidate          1.949715
Not decided                9.277395
dtype: float64
-
Result Dem Sen 4.0 (80% CI: -7.0 to 15.0) (Weighted N=1164) (raw_moe=3.7pts, margin=11.9pts, sigma=9.2pts) (Dem Sen 61.9% likely to win)
Dem 4.0 (80% CI: -7.0 to 15.0) (61.9% Dem)
-
-
## MISSISSIPPI ##
A Democratic candidate    40.652436
A Republican candidate    48.539188
Another candidate          1.684344
Not decided                9.124031
dtype: float64
-
Result Dem Sen -7.9 (80% CI: -18.7 to 3.1) (Weighted N=1164) (raw_moe=3.7pts, margin=11.7pts, sigma=9.1pts) (Dem Sen 27.1% likely to win)
Dem -7.9 (80% CI: -18.7 to 3.1) (27.1% Dem)
-
-
## MISSOURI ##
A Democratic candidate    40.652436
A Republican candidate    48.539188
Another candidate          1.684344
Not decided                9.124031
dtype: float64
-
Result Dem Sen -7.9 (80% CI: -18.7 to 3.0) (Weighted N=1164) (raw_moe=3.7pts, margin=11.7pts, sigma=9.1pts) (Dem S

Result Dem Sen 6.8 (80% CI: -4.2 to 17.7) (Weighted N=1164) (raw_moe=3.7pts, margin=11.7pts, sigma=9.2pts) (Dem Sen 69.7% likely to win)
Dem 6.8 (80% CI: -4.2 to 17.7) (69.7% Dem)
-
-
## WASHINGTON ##
A Democratic candidate    50.966536
A Republican candidate    37.741087
Another candidate          2.058224
Not decided                9.234153
dtype: float64
-
Result Dem Sen 13.2 (80% CI: 2.2 to 24.3) (Weighted N=1164) (raw_moe=3.7pts, margin=11.9pts, sigma=9.3pts) (Dem Sen 84.3% likely to win)
Dem 13.2 (80% CI: 2.2 to 24.3) (84.3% Dem)
-
-
## WEST VIRGINIA ##
A Democratic candidate    32.378668
A Republican candidate    56.378502
Another candidate          1.700973
Not decided                9.541856
dtype: float64
-
Result Dem Sen -24.0 (80% CI: -34.9 to -13.1) (Weighted N=1164) (raw_moe=3.6pts, margin=11.8pts, sigma=9.2pts) (Dem Sen 3.3% likely to win)
Dem -24.0 (80% CI: -34.9 to -13.1) (3.3% Dem)
-
-
## WISCONSIN ##
A Democratic candidate    45.874498
A Republican candidate    43.08